<a href="https://colab.research.google.com/github/ShivikaPrasanna/brain_cancer_gene_expression/blob/main/Braincancer_gene_expression_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

# Load the data
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/Brain_GSE50161.csv")

cancer_type_dict = {'ependymoma': 0, 'glioblastoma': 1, 'medulloblastoma': 2, 'pilocytic_astrocytoma': 3, 'normal': 4}
data['mapped_type'] = data['type'].replace(cancer_type_dict)
data.drop('type', axis=1, inplace=True)
data['samples'].groupby(data['mapped_type']).count()

labels = data.pop('mapped_type')
data = data.to_numpy().astype(np.float32)
labels = labels.to_numpy().flatten().astype(np.float32)

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min', restore_best_weights=True)
mcp_save = tf.keras.callbacks.ModelCheckpoint('my-model-{accuracy:.2f}.hdf5', save_best_only=True, monitor='accuracy', mode='max', verbose=1)
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                874832    
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 4)                 36        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 875,009
Trainable params: 875,009
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy*100:.2f}%")

Epoch 1/50
8/8 - 3s - loss: 1.5653 - accuracy: 0.4615 - 3s/epoch - 378ms/step
Epoch 2/50
8/8 - 0s - loss: -3.9088e+00 - accuracy: 0.5812 - 284ms/epoch - 36ms/step
Epoch 3/50
8/8 - 0s - loss: -1.6003e+01 - accuracy: 0.5385 - 338ms/epoch - 42ms/step
Epoch 4/50
8/8 - 0s - loss: -3.6402e+01 - accuracy: 0.4872 - 284ms/epoch - 35ms/step
Epoch 5/50
8/8 - 0s - loss: -7.1067e+01 - accuracy: 0.5128 - 321ms/epoch - 40ms/step
Epoch 6/50
8/8 - 0s - loss: -1.3090e+02 - accuracy: 0.4615 - 274ms/epoch - 34ms/step
Epoch 7/50
8/8 - 0s - loss: -2.2190e+02 - accuracy: 0.4872 - 287ms/epoch - 36ms/step
Epoch 8/50
8/8 - 0s - loss: -3.3151e+02 - accuracy: 0.4701 - 260ms/epoch - 32ms/step
Epoch 9/50
8/8 - 0s - loss: -4.7841e+02 - accuracy: 0.4530 - 395ms/epoch - 49ms/step
Epoch 10/50
8/8 - 0s - loss: -6.5235e+02 - accuracy: 0.4701 - 271ms/epoch - 34ms/step
Epoch 11/50
8/8 - 0s - loss: -8.5284e+02 - accuracy: 0.4615 - 278ms/epoch - 35ms/step
Epoch 12/50
8/8 - 0s - loss: -1.1215e+03 - accuracy: 0.4701 - 386ms/ep

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter( y=history.history['loss'], name="loss"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter( y=history.history['accuracy'], name="train accuracy"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Loss/Accuracy of Neural Network Model"
)

# Set x-axis title
fig.update_xaxes(title_text="Epoch")

In [ ]:
score, acc = model.evaluate(X_test, y_test, batch_size=32, verbose=2)
print("Accuracy: ", acc)

1/1 - 0s - loss: -1.3199e+05 - accuracy: 0.6154 - 99ms/epoch - 99ms/step
Accuracy:  0.6153846383094788
